In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob 
import os
import plotly.graph_objects as go
import plotly.express as px
import plotly
plotly.io.orca.config.executable = str(pathlib.Path().absolute()) + '/bins/orca'
print(plotly.io.orca.config.executable)

In [39]:
filenames = list(glob.iglob("../OUTPUT/Sink/*.csv"))
print(len(filenames))

720


In [25]:
pd_dataframes = {'continuous': [], 'phi': [], 'summary': [], 'positions': [], 'initialpositions': [], 'finished': [], 'lost': []}
for filename in filenames:
    full_path = os.path.abspath(filename)
    path = os.path.dirname(full_path)
    basename = os.path.basename(filename)
    finished_filename = path + "/" + "finished_" + "_".join(basename.split("_")[1:])
    #if not os.path.isfile(finished_filename):
    #    print("%s does not exist ! " % os.path.basename(finished_filename))
    #    print("%s does exist" % basename)
    #    continue
    try:
        data_file_delimiter = ','
        header = []
        with open(full_path, 'r') as temp_f:
            header = temp_f.readline().rstrip('\n').split(data_file_delimiter) + ['empty']
        df = pd.read_csv(full_path, names=header, skiprows=1)
        basename = os.path.basename(filename)
        name = basename.split("_")[0]
        arguments = basename.rstrip(".csv").split("_")[3:]
        it = iter(arguments)
        for val in it:
            parameter = val.split("=")[0]
            value = val.split("=")[1]
            df[parameter] = value
        pd_dataframes[name].append(df)
    except Exception as e:
        print(e)
        #print(full_path)
        print("Error: "+ full_path)

In [26]:
dfo = {}
for name in pd_dataframes:
    if(pd_dataframes[name]):
        dfo[name] = pd.concat(pd_dataframes[name])
df = dfo.copy()

In [27]:
def sourceAddressToAgentId(sa):
    try:
        return int(sa.split(".")[-1])-1
    except AttributeError:
        print(sa)
        return -1

df['continuous']['agent_id'] = df['continuous']['Source Address'].apply(sourceAddressToAgentId)
df['summary']['agent_id'] = df['summary']['Source Address'].apply(sourceAddressToAgentId)
df['continuous']['Throughput'] = (df['continuous']['Rx Bytes']*8)

In [28]:
common_columns = ['N', 'distance', 'wifiManager', 'simulationTime', 'agentType', 'dataRate', 'udp', 'RngRun', 'scenarioType', 'agent_id']
df['continuous'] = df['continuous'].set_index(common_columns)
df['positions'] = df['positions'].set_index(common_columns)
df['summary'] = df['summary'].set_index(common_columns)

In [29]:
df['continuous'].head()

Time  \
N distance wifiManager           simulationTime agentType dataRate udp  RngRun scenarioType agent_id         
5 100      ns3::IntelWifiManager 100            Client    50Mbps   true 14     1            1            3   
                                                                                            2            3   
                                                                                            3            3   
                                                                                            4            3   
                                                                                            1            4   

                                                                                                      Flow  \
N distance wifiManager           simulationTime agentType dataRate udp  RngRun scenarioType agent_id         
5 100      ns3::IntelWifiManager 100            Client    50Mbps   true 14     1            1            5   
                                                                                            2            6   
                                                                                            3            7   
                                                                                            4            8   
                                                                                            1            5   

                                                                                                     Source Address  \
N distance wifiManager           simulationTime agentType dataRate udp  RngRun scenarioType agent_id                  
5 100      ns3::IntelWifiManager 100            Client    50Mbps   true 14     1            1           192.168.1.2   
                                                                                            2           192.168.1.3   
                                                                                            3           192.168.1.4   
                                                                                            4           192.168.1.5   
                                                                                            1           192.168.1.2   

                                                                                                     Destination Address  \
N distance wifiManager           simulationTime agentType dataRate udp  RngRun scenarioType agent_id                       
5 100      ns3::IntelWifiManager 100            Client    50Mbps   true 14     1            1                192.168.1.1   
                                                                                            2                192.168.1.1   
                                                                                            3                192.168.1.1   
                                                                                            4                192.168.1.1   
                                                                                            1                192.168.1.1   

                                                                                                      Tx Packets  \
N distance wifiManager           simulationTime agentType dataRate udp  RngRun scenarioType agent_id               
5 100      ns3::IntelWifiManager 100            Client    50Mbps   true 14     1            1               6104   
                                                                                            2               6104   
                                                                                            3               6104   
                                                                                            4               6104   
                                                                                            1               6103   

                                            

In [30]:
df['positions'].head()

position_x  \
N distance wifiManager           simulationTime agentType dataRate udp  RngRun scenarioType agent_id               
5 100      ns3::IdealWifiManager 100            Client    50Mbps   true 18     1            0               0.00   
                                                                                            1             -15.69   
                                                                                            2             -24.21   
                                                                                            3             -96.76   
                                                                                            4              68.08   

                                                                                                      position_y  \
N distance wifiManager           simulationTime agentType dataRate udp  RngRun scenarioType agent_id               
5 100      ns3::IdealWifiManager 100            Client    50Mbps   true 18     1            0               0.00   
                                                                                            1             -98.76   
                                                                                            2              97.03   
                                                                                            3             -25.25   
                                                                                            4             -73.24   

                                                                                                      position_z  \
N distance wifiManager           simulationTime agentType dataRate udp  RngRun scenarioType agent_id               
5 100      ns3::IdealWifiManager 100            Client    50Mbps   true 18     1            0                  0   
                                                                                            1                  0   
                                                                                            2                  0   
                                                                                            3                  0   
                                                                                            4                  0   

                                                                                                      orientation_x  \
N distance wifiManager           simulationTime agentType dataRate udp  RngRun scenarioType agent_id                  
5 100      ns3::IdealWifiManager 100            Client    50Mbps   true 18     1            0                     0   
                                                                                            1                     0   
                                                                                            2                     0   
                                                                                            3                     0   
                                                                                            4                     0   

                                                                                                      orientation_y  \
N distance wifiManager           simulationTime agentType dataRate udp  RngRun scenarioType agent_id                  
5 100      ns3::IdealWifiManager 100            Client    50Mbps   true 18     1            0                     0   
                                                                                            1                     0   
                                                                                            2                     0   
                                                                                            3                     0   
                                                                                            4                     0

In [31]:
df['summary'].head()

Flow  \
N distance wifiManager                simulationTime agentType dataRate udp  RngRun scenarioType agent_id         
5 100      ns3::IntelWifiManager      100            Drone     50Mbps   true 6      1            1            5   
                                                                                                 2            6   
                                                                                                 3            7   
                                                                                                 4            8   
           ns3::MinstrelHtWifiManager 100            Drone     50Mbps   true 17     1            1            5   

                                                                                                          Source Address  \
N distance wifiManager                simulationTime agentType dataRate udp  RngRun scenarioType agent_id                  
5 100      ns3::IntelWifiManager      100            Drone     50Mbps   true 6      1            1           192.168.1.2   
                                                                                                 2           192.168.1.3   
                                                                                                 3           192.168.1.4   
                                                                                                 4           192.168.1.5   
           ns3::MinstrelHtWifiManager 100            Drone     50Mbps   true 17     1            1           192.168.1.2   

                                                                                                          Destination Address  \
N distance wifiManager                simulationTime agentType dataRate udp  RngRun scenarioType agent_id                       
5 100      ns3::IntelWifiManager      100            Drone     50Mbps   true 6      1            1                192.168.1.1   
                                                                                                 2                192.168.1.1   
                                                                                                 3                192.168.1.1   
                                                                                                 4                192.168.1.1   
           ns3::MinstrelHtWifiManager 100            Drone     50Mbps   true 17     1            1                192.168.1.1   

                                                                                                           Tx Packets  \
N distance wifiManager                simulationTime agentType dataRate udp  RngRun scenarioType agent_id               
5 100      ns3::IntelWifiManager      100            Drone     50Mbps   true 6      1            1             610351   
                                                                                                 2             610351   
                                                                                                 3             610351   
                                                                                                 4             610351   
           ns3::MinstrelHtWifiManager 100            Drone     50Mbps   true 17     1            1             610351   

                                                                                                            Tx Bytes  \
N distance wifiManager                simulationTime agentType dataRate udp  RngRun scenarioType agent_id              
5 100      ns3::IntelWifiManager      100            Drone     50Mbps   true 6      1            1         642089252   
                                                                                                 2         642089252   
                                                                                                 3         642089252   
                                                                                

In [32]:
new_df = df['continuous'].join(df['positions'], how='left', on=common_columns, lsuffix='_left', rsuffix='_right')
common_columns += ['Source Address', 'Destination Address']
new_df = new_df.reset_index().set_index(common_columns)
df['summary'] = df['summary'].reset_index().set_index(common_columns)
new_df = new_df.join(df['summary'], how='left', on=common_columns, rsuffix='_total')
new_df = new_df.reset_index()

In [33]:
new_df.head()

,N,distance,wifiManager,simulationTime,agentType,dataRate,udp,RngRun,scenarioType,agent_id,...,orientation_w,angle,last_change,empty_right,Flow_total,Tx Packets_total,Tx Bytes_total,Rx Packets_total,Rx Bytes_total,empty
0,5,100,ns3::IntelWifiManager,100,Client,50Mbps,true,14,1,1,...,-0.86,299,33.60,NaN,5,610351,642089252,210807,221768964,NaN
1,5,100,ns3::IntelWifiManager,100,Client,50Mbps,true,14,1,2,...,0.02,178,27.67,NaN,6,610351,642089252,450247,473659844,NaN
2,5,100,ns3::IntelWifiManager,100,Client,50Mbps,true,14,1,3,...,0.30,145,29.05,NaN,7,610351,642089252,443183,466228516,NaN
3,5,100,ns3::IntelWifiManager,100,Client,50Mbps,true,14,1,4,...,1.00,355,35.05,NaN,8,610351,642089252,241535,254094820,NaN
4,5,100,ns3::IntelWifiManager,100,Client,50Mbps,true,14,1,1,...,-0.86,299,33.60,NaN,5,610351,642089252,210807,221768964,NaN


In [34]:
convergence_time = []
g = None
for name, group in new_df.groupby(['N', 'distance', 'wifiManager', 'simulationTime', 'agentType', 'dataRate', 'udp', 'RngRun']):
    local_data = []
    antenna_convergence = group['last_change'].max()
    throughput_convergence = None
    full_rx = None
    g = group
    for agent_id, agent_group in group.groupby(['agent_id']):
        low, high = 0.95*(agent_group['Rx Bytes'].values[-1]), 1.05*(agent_group['Rx Bytes'].values[-1])
        agent_group['between'] = agent_group['Rx Bytes'].between(low, high)
        agent_group['between'] = ~agent_group['between']
        k = agent_group['between'][agent_group['between']].index.max()
        #k = agent_group['between'][agent_group['between']==False].last_valid_index()
        if (k is None) or (pd.isna(k)):
            throughput_convergence = 0
        else:
            throughput_convergence = agent_group.loc[k]['Time']
        if name[4] == "Drone":
            full_rx = agent_group[agent_group['Time'] >= antenna_convergence]['Throughput'].mean()
        else:
            full_rx = agent_group['Throughput'].mean()
        convergence_time.append(list(name) + [agent_id, throughput_convergence, antenna_convergence, full_rx])

In [35]:
conv = pd.DataFrame(convergence_time, columns=['N', 'distance', 'wifiManager', 'simulationTime', 'agentType', 'dataRate', 'udp', 'RngRun', 'agent_id', 'Throughput Converge', 'Antenna Converge', 'Throughput'])
conv = conv.groupby(['N', 'distance', 'wifiManager', 'simulationTime', 'agentType', 'dataRate', 'udp', 'RngRun']).mean().reset_index()
full = conv.copy()
conv = conv[conv['agentType'] == 'Drone']

In [36]:
radius = '100'
filtered = conv[(conv['distance'] == radius)]
filtered = filtered[(filtered['dataRate'] == '50Mbps')]
filtered = filtered[(filtered['N'] == '5')]

trace0 = go.Box(
    y=filtered['Antenna Converge'],
    x=filtered['wifiManager'],
    name='Orientation Convergence Time',
    boxmean=True
)

trace1 = go.Box(
    y=filtered['Throughput Converge'],
    x=filtered['wifiManager'],
    name='Throughput Convergence Time',
    boxmean=True
)

data = [trace0, trace1]
layout = go.Layout(
    #title = "foo",
    template = "simple_white",
    xaxis_title="Wifi Manager",
    yaxis=go.layout.YAxis(
        title='Time (s)',
        range=[-5, max(filtered['Throughput Converge'].max()*1.05, filtered['Antenna Converge'].max()*1.05)],
        zeroline=False
    ),
    boxmode='group'
)

fig = go.Figure(data=data, layout=layout)
fig.update_layout(legend_orientation="h")
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.show()
fig.write_image("convergence_sink_3dB.png")

In [37]:
radius = '100'
filtered = full[(full['distance'] == radius)]
print(set(filtered['N']))
filtered = filtered[(filtered['N'] == '5')]
filtered = filtered[(filtered['dataRate'] == '50Mbps')]
drone = filtered[(filtered['agentType'] == 'Drone')]
client = filtered[(filtered['agentType'] == 'Client')]

trace0 = go.Box(
    y=(client['Throughput']),
    x=client['wifiManager'],
    name='Client',
    boxmean=True
)

trace1 = go.Box(
    y=(drone['Throughput']),
    x=drone['wifiManager'],
    name='Drone',
    boxmean=True
)


data = [trace0, trace1]
layout = go.Layout(
    template = "simple_white",
    xaxis_title="Wifi Manager",
    yaxis=go.layout.YAxis(
        title='Throughput (bps)',
    ),
    boxmode='group'
)

fig = go.Figure(data=data, layout=layout)
fig.update_layout(legend_orientation="h")
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.show()
fig.write_image("débit_sink_3dB.png")

{'5'}


In [16]:
full[full['Throughput'] <= 1000]

,N,distance,wifiManager,simulationTime,agentType,dataRate,udp,RngRun,agent_id,Throughput Converge,Antenna Converge,Throughput


In [17]:
full[full['agentType'] == "Drone"].head()

,N,distance,wifiManager,simulationTime,agentType,dataRate,udp,RngRun,agent_id,Throughput Converge,Antenna Converge,Throughput
20,5,100,ns3::IdealWifiManager,100,Drone,50Mbps,true,0,2.5,99.75,23.97,2.857704e+07
21,5,100,ns3::IdealWifiManager,100,Drone,50Mbps,true,1,2.5,100.00,25.49,2.886148e+07
22,5,100,ns3::IdealWifiManager,100,Drone,50Mbps,true,10,2.5,100.00,23.60,2.861170e+07
23,5,100,ns3::IdealWifiManager,100,Drone,50Mbps,true,11,2.5,100.00,24.46,2.886945e+07
24,5,100,ns3::IdealWifiManager,100,Drone,50Mbps,true,12,2.5,100.00,25.15,2.855449e+07


In [18]:
df['summary'] = df['summary'].reset_index()
df['summary']['Throughput'] = (df['summary']['Rx Bytes']*8) / pd.to_numeric(df['summary']['simulationTime'])
mads = []
for name, group in df['summary'].groupby(['N', 'distance', 'wifiManager', 'simulationTime', 'agentType', 'dataRate', 'udp', 'RngRun']):
    jain = ((group['Throughput'].sum())**2)/((len(group['Throughput'].index))*(group['Throughput']*group['Throughput']).sum())
    mads.append(list(name) + [group['Throughput'].mean(), group['Throughput'].mad(), jain])
mads = pd.DataFrame(mads, columns=['N', 'distance', 'wifiManager', 'simulationTime', 'agentType', 'dataRate', 'udp', 'RngRun', 'Throughput', 'MAD', 'Jain'])

In [19]:
a = mads.copy()
a['distance'] = pd.to_numeric(a['distance'])
a['dataRate'] = pd.to_numeric(a['dataRate'].str.rstrip("Mbps"))
a['N'] = pd.to_numeric(a['N'])
a = a[a['distance'] == 100]
a = a[a['N'] == 5]
a.sort_values(by=['wifiManager', 'dataRate', 'agentType', 'distance'], inplace=True)
a['cross'] = a['wifiManager'] + " " + a['dataRate'].apply(lambda x: str(x))

drone = a[(a['agentType'] == 'Drone')]
client = a[(a['agentType'] == 'Client')]

trace0 = go.Box(
    y=(client['Jain']),
    x=client['wifiManager'],
    name='Client',
    boxmean=True
)

trace1 = go.Box(
    y=(drone['Jain']),
    x=drone['wifiManager'],
    name='Drone',
    boxmean=True
)


data = [trace0, trace1]
layout = go.Layout(
    template = "simple_white",
    xaxis_title="Wifi Manager",
    yaxis=go.layout.YAxis(
        title='Jain index',
    ),
    boxmode='group'
)

fig = go.Figure(data=data, layout=layout)
fig.update_layout(legend_orientation="h")
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.show()

In [20]:
radius = '100'
filtered = full[(full['distance'] == radius)]
filtered = filtered[(filtered['N'] == '5')]
filtered['cross'] = filtered['wifiManager'] + " " + filtered['dataRate'].apply(lambda x: str('{num:03d}'.format(num=int(x.rstrip("Mbps")))))
filtered = filtered.sort_values(by=['cross'])
drone = filtered[(filtered['agentType'] == 'Drone')]
client = filtered[(filtered['agentType'] == 'Client')]


fig = go.Figure(data=data, layout=layout)

trace0 = go.Box(
    y=(client['Throughput']),
    x=client['cross'],
    name='Isotropic Antenna',
    boxmean=True
)

trace1 = go.Box(
    y=(drone['Throughput']),
    x=drone['cross'],
    name='AP-AC-Mesh Antenna',
    boxmean=True
)


data = [trace0, trace1]
layout = go.Layout(
    template = "simple_white",
    xaxis_title="Wifi Manager",
    yaxis=go.layout.YAxis(
        title='Throughput (bps)',
    ),
    boxmode='group'
)

fig = go.Figure(data=data, layout=layout)
fig.update_layout(legend_orientation="h")
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.show()
fig.write_image("100m_sink_comparison_3dB.png")

In [21]:
set(mads['distance'])

{'100'}

In [22]:
pd.options.display.float_format = "{:.2f}".format
mads['N'] = pd.to_numeric(mads['N'])
a=mads.copy()
a=a[a['dataRate'] == '50Mbps']
a.groupby(['N', 'distance', 'wifiManager', 'dataRate', 'simulationTime', 'agentType']).describe()['Jain'].reset_index()

,N,distance,wifiManager,dataRate,simulationTime,agentType,count,mean,std,min,25%,50%,75%,max
0,5,100,ns3::IdealWifiManager,50Mbps,100,Client,20.00,0.92,0.06,0.82,0.87,0.92,0.99,1.00
1,5,100,ns3::IdealWifiManager,50Mbps,100,Drone,20.00,0.95,0.05,0.86,0.92,0.97,0.99,1.00
2,5,100,ns3::IntelWifiManager,50Mbps,100,Client,20.00,0.93,0.05,0.87,0.89,0.92,0.98,1.00
3,5,100,ns3::IntelWifiManager,50Mbps,100,Drone,20.00,0.95,0.06,0.83,0.90,0.97,0.99,1.00
4,5,100,ns3::MinstrelHtWifiManager,50Mbps,100,Client,20.00,0.96,0.03,0.90,0.95,0.97,0.99,1.00
5,5,100,ns3::MinstrelHtWifiManager,50Mbps,100,Drone,20.00,0.96,0.03,0.88,0.95,0.97,0.98,1.00
